In [1]:
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x104014818>>

In [8]:
for floor in range(0,5):
    for building in range(0,3):
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        if (data_num == 0):
            continue
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        user_list = np.unique(wifi_loc_time[:, 522])
        user_num = len(user_list)
        print("There are ", user_num, " users.")
        
        longitude_list = sorted(np.unique(wifi_loc_time[:, 520]))
        latitude_list = sorted(np.unique(wifi_loc_time[:, 521]))
        
        long_dict = {}
        loc_dict= {}
        for i in range(len(longitude_list)):
            long_dict[float(longitude_list[i])] = 0
        for i in range(len(latitude_list)):
            loc_dict[float(latitude_list[i])] = long_dict.copy()

        user_walking_num = np.zeros(shape=(user_num), dtype=int)
        for i in range(len(wifi_loc_time[:,522])):
            for user_index in range(user_num):
                if (wifi_loc_time[i, 522] == user_list[user_index]):
                    user_walking_num[user_index] += 1
        
        seq_window = 9
        total_batch_num = 0
        
        for num in user_walking_num:
            total_batch_num += math.floor(num/seq_window)
        print("total_batch_num: ",total_batch_num)
        
        batch_wifi = np.zeros((total_batch_num, seq_window, 520))
        batch_loc = np.zeros((total_batch_num, seq_window), dtype = int)
        
        state_count = 0
        batch_num = 0
        iter_batch = 0
        iter_window = 0
        start_location = np.zeros(shape=(2))
        
        for k in range(len(user_list)):
            print("User",k, " walking ", user_walking_num[k])
            user_fp = np.zeros((user_walking_num[k], 2+520))
            batch_num += math.floor(user_walking_num[k]/9)
            user_i = 0
            for i in range(len(wifi_loc_time[:,522])):
                if (wifi_loc_time[i, 522] == user_list[k]):
                    user_fp[user_i, :2] = np.array(wifi_loc_time[i, 520:522])
                    user_fp[user_i, 2:] = np.array(wifi_loc_time[i, :520])
                    user_i += 1
        
            if (state_count == 0):
                # find the start
                for i in range(len(latitude_list)):
                    for j in range(len(longitude_list)):
                        if (user_fp[0][0] == longitude_list[j] and user_fp[0][1] == latitude_list[i]):
                            if (loc_dict[latitude_list[i]][longitude_list[j]] == 0):
                                loc_dict[latitude_list[i]][longitude_list[j]] = state_count
                                start_location[0] = i
                                start_location[1] = j
                                state_count += 1
                                print("Start state location:", floor, building, i, j, loc_dict[latitude_list[i]][longitude_list[j]])
                                break
                     
                for s in range(1,math.floor(user_walking_num[k]/9)*9):
                    for i in range(len(latitude_list)):
                        for j in range(len(longitude_list)):
                            if (user_fp[s][0] == longitude_list[j] and user_fp[s][1] == latitude_list[i]):
                                if (loc_dict[latitude_list[i]][longitude_list[j]] == 0 and (i!=start_location[0] and j!=start_location[1])):
                                    loc_dict[latitude_list[i]][longitude_list[j]] = state_count
                                    state_count = state_count + 1
                                #print(s, i, j, loc_dict[latitude_list[i]][longitude_list[j]])
            else:
                for s in range(math.floor(user_walking_num[k]/9)*9):
                    for i in range(len(latitude_list)):
                        for j in range(len(longitude_list)):
                            if (user_fp[s][0] == longitude_list[j] and user_fp[s][1] == latitude_list[i]):
                                if (loc_dict[latitude_list[i]][longitude_list[j]] == 0 and (i!=start_location[0] and j!=start_location[1])):
                                    loc_dict[latitude_list[i]][longitude_list[j]] = state_count
                                    state_count = state_count + 1
                                #print(s, i, j, loc_dict[latitude_list[i]][longitude_list[j]])
                                    
            for i in range(math.floor(user_walking_num[k]/seq_window)*seq_window):
                batch_wifi[iter_batch][iter_window] = user_fp[i][2:]
                batch_loc[iter_batch,iter_window] = loc_dict[float(user_fp[i][1])][float(user_fp[i][0])]
                #print(iter_batch, iter_window, batch_loc[iter_batch,iter_window])
                if (iter_window == (seq_window-1)):
                    iter_window = 0
                    iter_batch = iter_batch + 1
                    if (iter_batch == (batch_num)):
                        break
                else:
                    iter_window = iter_window + 1
                    
        # Testing
        # extract data
        data_num_v = 0 # 86 in validation dataset
        with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
            spamreader_v = csv.reader(v_csvfile, delimiter=',')
            for row in spamreader_v:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num_v += 1

        print(data_num_v)
        if (data_num_v == 0):
            continue
        i = -1
        wifi_loc_time_v = np.zeros(shape = (data_num_v, 524))
        
        with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
            spamreader_v = csv.reader(v_csvfile, delimiter=',')
            for row in spamreader_v:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                #if (i > data_num):
                #    break
                # wifi
                wifi_loc_time_v[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time_v[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time_v[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time_v[i-1][-1] = np.array(row[-1])
                
        wifi_loc_time_v = wifi_loc_time_v[wifi_loc_time_v[:,-1].argsort()]
        longitude_list_v = sorted(np.unique(wifi_loc_time_v[:, 520:521]))
        latitude_list_v = sorted(np.unique(wifi_loc_time_v[:, 521:522]))
        
        long_dict_v = {}
        loc_dict_v = {}
        for i in range(len(longitude_list_v)):
            long_dict_v[float(longitude_list_v[i])] = i
        for i in range(len(latitude_list_v)):
            loc_dict_v[float(latitude_list_v[i])] = long_dict_v.copy()
                
        min_dis = 0
        max_ind = -1
        distance  = 0
        cumulative_distance = 0

        # find the nearest loc_dict_v
        for i in range(len(latitude_list_v)):
            for j in range(len(longitude_list_v)):
                min_dis = sys.float_info.max
                max_ind = -1
                for x in range(len(latitude_list)):
                    for y in range(len(longitude_list)):
                        distance = math.sqrt(pow(latitude_list[x]-latitude_list_v[i],2) \
                                             +pow(longitude_list[y]-longitude_list_v[j], 2))
                        if (min_dis > distance):
                            if (loc_dict[float(latitude_list[x])][float(longitude_list[y])] != 0 or \
                               (loc_dict[float(latitude_list[x])][float(longitude_list[y])] == 0\
                                      and x == 65 and y == 11)):
                                min_dis = distance
                                max_ind = loc_dict[float(latitude_list[x])][float(longitude_list[y])]
                cumulative_distance = cumulative_distance + min_dis
                loc_dict_v[latitude_list_v[i]][longitude_list_v[j]] = max_ind
        print("GT & map RMSE: ", cumulative_distance/(len(latitude_list_v)*len(longitude_list_v)))        
                
        batch_num = int(data_num_v/seq_window) # 3 * 28 = 84
        print("Validating data", batch_num)
        batch_wifi_v = np.zeros((batch_num + 1, seq_window, 520))
        batch_loc_v = np.zeros((batch_num + 1, seq_window), dtype = int)

        #batch_wifi_v[0][0] = batch_wifi[0]
        #batch_loc_v[0,0] = loc_dict[float(latitude_list[start_location[0]])][float(longitude_list[start_location[1]])]

        iter_window = 0
        iter_batch = 1

        for i in range(len(wifi_loc_time_v)):
            batch_wifi_v[iter_batch][iter_window] = wifi_loc_time_v[i][:520]
            batch_loc_v[iter_batch,iter_window] = loc_dict_v[float(wifi_loc_time_v[i][521])][float(wifi_loc_time_v[i][520])]
            if (iter_window == (seq_window-1)):
                iter_window = 0
                if (iter_batch < (batch_num)):
                    iter_batch = iter_batch + 1
                elif (iter_batch == (batch_num)):
                    iter_batch = iter_batch + 1
                    break
            else:
                iter_window = iter_window + 1

#         for i in range(batch_num+1):
#             print(i, batch_loc_v[i])        
                

        print(batch_loc)
        model = ChainCRF()
        ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=80)            
        ssvm.fit(batch_wifi, batch_loc)
        
        pred_batch_v = ssvm.predict(batch_wifi_v)
        total_error = 0
        total_error_loc_dict_space = 0

        validation_latitude = []
        validation_longitude = []

        gt_validation_latitude_loc_dict_space = []
        gt_validation_longitude_loc_dict_space = []
        iter_validating_data = -1
        cumulative_distance = 0
        for i in range(1,batch_num+1):
            for j in range(seq_window):
                if (batch_loc_v[i][j] == 0):
                    gt_validation_latitude_loc_dict_space.append(latitude_list[int(start_location[0])])
                    gt_validation_longitude_loc_dict_space.append(longitude_list[int(start_location[1])])
                    continue
                iter_validating_data+=1
                for x in range(len(latitude_list)):
                    for y in range(len(longitude_list)):
                        if (batch_loc_v[i][j] == loc_dict[latitude_list[x]][longitude_list[y]]):
                            gt_validation_latitude_loc_dict_space.append(latitude_list[x])
                            gt_validation_longitude_loc_dict_space.append(longitude_list[y])
                            #print("iter_validating_data", iter_validating_data)
                            cumulative_distance += math.sqrt(pow(latitude_list[x]-wifi_loc_time_v[iter_validating_data][521],2)+pow(longitude_list[y]-wifi_loc_time_v[iter_validating_data][520],2))
                            break
        print("cumulative_distance is :", cumulative_distance/(seq_window*batch_num))

        iter_validating_data = 0
        window = np.zeros((seq_window))
        for i in range(1,batch_num+1):
            for j in range(seq_window):
                for x in range(len(latitude_list)):
                    for y in range(len(longitude_list)):
                        if (pred_batch_v[i][j] == loc_dict[latitude_list[x]][longitude_list[y]]):
                            validation_latitude.append(latitude_list[x])
                            validation_longitude.append(longitude_list[y])
                            total_error += math.sqrt(pow(latitude_list[x] - wifi_loc_time_v[iter_validating_data][521],2)+pow(longitude_list[y] - wifi_loc_time_v[iter_validating_data][520],2))
                            #print(iter_validating_data, i, j)
                            #print("state:", loc_dict[latitude_list[x]][longitude_list[y]], batch_loc_v[i][j], "       Distance:",math.sqrt(pow(latitude_list[x] - wifi_loc_time_v[iter_validating_data][521],2)+pow(longitude_list[y] - wifi_loc_time_v[iter_validating_data][520],2)))
                            break
                window[j] += math.sqrt(pow(validation_latitude[iter_validating_data] - gt_validation_latitude_loc_dict_space[iter_validating_data],2)+pow(validation_longitude[iter_validating_data] - gt_validation_longitude_loc_dict_space[iter_validating_data],2))
                total_error_loc_dict_space += math.sqrt(pow(validation_latitude[iter_validating_data] - gt_validation_latitude_loc_dict_space[iter_validating_data],2)+pow(validation_longitude[iter_validating_data] - gt_validation_longitude_loc_dict_space[iter_validating_data],2))
                iter_validating_data += 1

        print("RMSE: ", total_error/(batch_num*seq_window))
        print("RMSE in loc_dict space:", total_error_loc_dict_space/(batch_num*seq_window))
        print("Windows:")
        for x in range(seq_window):
            print(x, window[x]/batch_num)
        print("Finish\n")

1059
There are  2  users.
total_batch_num:  117
User 0  walking  528
Start state location: 0 0 10 0 0
User 1  walking  531
78
GT & map RMSE:  13.216316922205563
Validating data 8
[[ 0  0  0 ...  0  0  0]
 [ 0  1  1 ...  1  1  1]
 [ 1  1  2 ...  2  2  2]
 ...
 [18 18 18 ... 18 19 19]
 [19 19 19 ... 19 19 21]
 [21 21 21 ... 21 21 21]]
cumulative_distance is : 1.7838596252968246
RMSE:  40.01168495286074
RMSE in loc_dict space: 49.29029283701834
Windows:
0 49.04870446495297
1 46.872198469489874
2 52.223069657734456
3 44.417850972420545
4 47.909267550378765
5 49.95687096629786
6 52.947104148137186
7 48.23184540232703
8 52.00572390142627
Finish

1368
There are  5  users.
total_batch_num:  150
User 0  walking  550
Start state location: 0 1 26 20 0
User 1  walking  139
User 2  walking  69
User 3  walking  138
User 4  walking  472
30
GT & map RMSE:  10.247068218169934
Validating data 3
[[ 0  0  0 ...  0  0  0]
 [ 0  1  1 ...  1  1  1]
 [ 1  1  2 ...  2  2  2]
 ...
 [73 74 74 ... 74 74 74]
 [74 

cumulative_distance is : 2.6915100090722
RMSE:  276.04352274656543
RMSE in loc_dict space: 30.53161992535432
Windows:
0 29.262616211523902
1 36.374723649833456
2 36.94300355995365
3 26.543833473574455
4 29.40253703461193
5 30.6752072415893
6 24.46474079523236
7 14.47177991463666
8 46.64613744723322
Finish

2709
There are  5  users.
total_batch_num:  298
User 0  walking  619
Start state location: 3 2 26 94 0
User 1  walking  428
User 2  walking  510
User 3  walking  613
User 4  walking  539
40
GT & map RMSE:  11.084403026763832
Validating data 4
[[ 0  0  0 ...  0  0  0]
 [ 0  1  1 ...  1  1  1]
 [ 1  1  2 ...  2  2  2]
 ...
 [70 70 70 ... 61 61 61]
 [61 61 61 ... 60 60 60]
 [60 60 60 ... 60 62 62]]
cumulative_distance is : 1.023142355020808
RMSE:  27.646278317968672
RMSE in loc_dict space: 27.726526460946353
Windows:
0 10.753726965784939
1 19.400008067319646
2 24.349885023322525
3 30.843630930755257
4 34.515965023793
5 25.4767839040593
6 31.668104353736833
7 31.879311636850783
8 40.6513